In [1]:
# Install necessary packages (run these if not already installed)
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install prettytable
!pip install -q pandas


     ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
     --- ------------------------------------ 0.5/6.0 MB 2.1 MB/s eta 0:00:03
     ------- -------------------------------- 1.0/6.0 MB 2.3 MB/s eta 0:00:03
     -------- ------------------------------- 1.3/6.0 MB 2.0 MB/s eta 0:00:03
     -------- ------------------------------- 1.3/6.0 MB 2.0 MB/s eta 0:00:03
     -------- ------------------------------- 1.3/6.0 MB 2.0 MB/s eta 0:00:03
     ---------- ----------------------------- 1.6/6.0 MB 1.0 MB/s eta 0:00:05
     ---------- ----------------------------- 1.6/6.0 MB 1.0 MB/s eta 0:00:05
     ------------ --------------------------- 1.8/6.0 MB 949.8 kB/s eta 0:00:05
     --------------

  DEPRECATION: Building 'sqlalchemy' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'sqlalchemy'. Discussion can be found at https://github.com/pypa/pip/issues/6334


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 3.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 3.6 MB/s eta 0:00:00

   ---------------------------------------- 0/6 [ipython-genutils]
   ------ --------------------------------- 1/6 [sqlparse]
   ------ --------------------------------- 1/6 [sqlparse]
   ------------- -------------------------- 2/6 [prettytable]
   -------------------- ------------------- 3/6 [greenlet]
   -------------------- ------------------- 3/6 [greenlet]
   -------------------- ------------------- 3/6 [greenlet]
  Attempting uninstall: sqlalchemy
   -------------------- ------------------- 3/6 [greenlet]
    Found existing installation: SQLAlchemy 1.3.9
   -------------------- ------------------- 3/6 [greenlet]
   -------------------------- ------------- 4/6 [sqlalchemy]
   -----------

In [ ]:
# Load SQL extension and import required modules
%load_ext sql

import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

# Create a SQLite database connection
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

# Connect to the SQLite database in Jupyter using SQL magic
%sql sqlite:///my_data1.db


In [21]:
# Load your dataset saved from the last lab (dataset_part_2.csv)
import pandas as pd
df = pd.read_csv("dataset_part_2.csv")


# Rename columns: remove spaces, lowercase, and use underscores
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('.', '', regex=False)

# Preview updated column names
print(df.columns)

# Save clean DataFrame to SQLite
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")



Index(['flight_no', 'launch_site', 'payload', 'payload_mass', 'orbit',
       'customer', 'launch_outcome', 'version_booster', 'booster_landing',
       'date', 'time', 'class'],
      dtype='object')


121

In [32]:
%load_ext sql
%sql sqlite:///my_data1.db


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [22]:
# Drop any existing cleaned table to avoid conflicts
%sql DROP TABLE IF EXISTS SPACEXTABLE;


 * sqlite:///my_data1.db
Done.


[]

In [23]:
 # Create a cleaned version of the dataset excluding blank rows
%sql CREATE TABLE SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL;


 * sqlite:///my_data1.db
Done.


[]

In [26]:
%sql SELECT DISTINCT launch_site FROM SPACEXTABLE;


 * sqlite:///my_data1.db
Done.


launch_site
CCAFS
VAFB
Cape Canaveral
KSC
CCSFS


In [27]:
%sql SELECT * FROM SPACEXTABLE WHERE launch_site LIKE 'CCA%' LIMIT 5;


 * sqlite:///my_data1.db
Done.


flight_no,launch_site,payload,payload_mass,orbit,customer,launch_outcome,version_booster,booster_landing,date,time,class
1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.07B0003.18,Failure,4 June 2010,18:45,1
2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43,1
3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt,22 May 2012,07:44,1
4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,F9 v1.07B0006.18,No attempt,8 October 2012,00:35,0
5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,F9 v1.07B0007.18,No attempt,1 March 2013,15:10,1


In [28]:
%sql SELECT SUM(payload_mass) AS total_payload FROM SPACEXTABLE WHERE customer = 'NASA (CRS)';


 * sqlite:///my_data1.db
Done.


total_payload
None


In [29]:
%sql SELECT AVG(payload_mass) AS avg_payload FROM SPACEXTABLE WHERE version_booster = 'F9 v1.1';


 * sqlite:///my_data1.db
Done.


avg_payload
2.6


In [30]:
%sql SELECT MIN(date) AS first_success_ground_pad FROM SPACEXTABLE WHERE booster_landing = 'Success (ground pad)';


 * sqlite:///my_data1.db
Done.


first_success_ground_pad
None


In [34]:
%%sql
SELECT version_booster 
FROM SPACEXTABLE 
WHERE booster_landing = 'Success (drone ship)' 
  AND CAST(payload_mass AS INTEGER) > 4000 
  AND CAST(payload_mass AS INTEGER) < 6000;


 * sqlite:///my_data1.db
Done.


version_booster


In [36]:
%%sql 
SELECT launch_outcome, COUNT(*) AS outcome_count 
FROM SPACEXTABLE 
GROUP BY launch_outcome;


 * sqlite:///my_data1.db
Done.


launch_outcome,outcome_count
Failure,1
Success,32
Success,88


In [37]:
%%sql
SELECT version_booster, payload_mass 
FROM SPACEXTABLE 
WHERE CAST(payload_mass AS INTEGER) = (
    SELECT MAX(CAST(payload_mass AS INTEGER)) FROM SPACEXTABLE
);


 * sqlite:///my_data1.db
Done.


version_booster,payload_mass
F9 v1.1[,570 kg


In [39]:
%%sql
SELECT 
    SUBSTR(date, 6, 2) AS month,
    booster_landing,
    version_booster,
    launch_site
FROM SPACEXTABLE
WHERE booster_landing = 'Failure (drone ship)'
  AND SUBSTR(date, 1, 4) = '2015';


 * sqlite:///my_data1.db
Done.


month,booster_landing,version_booster,launch_site


In [40]:
%%sql
SELECT booster_landing, COUNT(*) AS outcome_count
FROM SPACEXTABLE
WHERE date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY booster_landing
ORDER BY outcome_count DESC;


 * sqlite:///my_data1.db
Done.


booster_landing,outcome_count
